In [1]:
import os
from langchain_openai.embeddings import OpenAIEmbeddings
from openai import OpenAI
from langchain_chroma.vectorstores import Chroma
from glob import glob
import json

In [2]:
from dotenv import load_dotenv
load_dotenv()
HF_TOKEN = os.getenv('HF_TOKEN')

In [3]:
client = OpenAI()
emb = OpenAIEmbeddings( model="text-embedding-3-small" )

In [4]:
text_files = glob('./data/*.txt')

In [5]:
PERSIST_DIR = "./db"  # 경로 혼동 방지(절대경로 추천)

vector_store_youth = Chroma(
    collection_name="YOUTH",     # 처음과 동일
    persist_directory=PERSIST_DIR,  # 처음과 동일
    embedding_function=emb,         # 동일 모델/차원
)

In [6]:
vector_store_marry1 = Chroma(
    collection_name="MARRY1",     # 처음과 동일
    persist_directory=PERSIST_DIR,  # 처음과 동일
    embedding_function=emb,         # 동일 모델/차원
)

In [7]:
vector_store_marry2 = Chroma(
    collection_name="MARRY2",     # 처음과 동일
    persist_directory=PERSIST_DIR,  # 처음과 동일
    embedding_function=emb,         # 동일 모델/차원
)

In [8]:
print(text_files)

['./data\\25년3차_신혼·신생아매입임대Ⅰ_공급주택목록(서울지역본부).txt', './data\\25년3차_신혼·신생아매입임대Ⅱ(전세형)_공급주택목록(서울지역본부).txt', './data\\25년_3차_청년매입임대_공급주택목록(서울지역본부).txt']


In [9]:
for file in text_files:
    texts, metadatas = [], []
    with open(file, 'r', encoding = 'utf-8') as f:
        s = f.read()
        data = json.loads(s)
        for item in data:
            info = item["metadata"]
            text = ", ".join(f"{k}: {v}" for k, v in info.items())
            info['source'] = file
            metadatas.append(info)
            texts.append(text)

    if file == './data\\25년3차_신혼·신생아매입임대Ⅰ_공급주택목록(서울지역본부).txt':
        vector_store_marry1.add_texts(texts, metadatas = metadatas)

    elif file == './data\\25년3차_신혼·신생아매입임대Ⅱ_공급주택목록(서울지역본부).txt':
        vector_store_marry2.add_texts(texts, metadatas = metadatas)

    elif file == './data\\25년_3차_청년매입임대_공급주택목록(서울지역본부).txt':
        vector_store_youth.add_texts(texts, metadatas = metadatas)

In [10]:
def search_with_filter(query: str, filters: dict, k: int = 5):
    return vector_store_youth.similarity_search(query, k=k, filter=filters)

docs = search_with_filter(
    "주소 정보 알려줘",
    {"구": "강서구"},
    k=5
)

In [11]:
print(docs)

[Document(id='e4763afb-6b25-4bcb-b739-f6eff7020d8b', metadata={'방수': 1, '주소': '서울특별시 강서구 화곡로 350(화곡동) SJ라벨라', '주택유형': '오피스텔', '청년 2순위, 3순위 여부': 'N', '호': 1516, '동': '화곡동', '면적계': 33.84, '구': '강서구', '주거공용 면적': 13.46, '층수': '15층', '승강기 유무': 'Y', '월임대료': 358480, 'source': './data\\25년_3차_청년매입임대_공급주택목록(서울지역본부).txt', '전용 면적': 20.38, '주택군 이름': '강서화곡동(SJ라벨라)', '청년1순위 여부': 'Y', '임대보증금': 1000000}, page_content='주소: 서울특별시 강서구 화곡로 350(화곡동) SJ라벨라, 구: 강서구, 동: 화곡동, 호: 1516, 주택군 이름: 강서화곡동(SJ라벨라), 전용 면적: 20.38, 주거공용 면적: 13.46, 면적계: 33.84, 방수: 1, 층수: 15층, 승강기 유무: Y, 주택유형: 오피스텔, 임대보증금: 1000000, 월임대료: 358480, 청년1순위 여부: Y, 청년 2순위, 3순위 여부: N'), Document(id='ee4eb320-1ecb-47f4-a087-dc15a25b1ef5', metadata={'임대보증금': 2000000, '동': '화곡동', '주택군 이름': '강서화곡동(SJ라벨라)', '호': 1516, '청년 2순위, 3순위 여부': 'Y', '주소': '서울특별시 강서구 화곡로 350(화곡동) SJ라벨라', '승강기 유무': 'Y', '월임대료': 439590, '전용 면적': 20.38, '주택유형': '오피스텔', '주거공용 면적': 13.46, '청년1순위 여부': 'N', '층수': '15층', 'source': './data\\25년_3차_청년매입임대_공급주택목록(서울지역본부).txt', '면적계': 33.84,